### Task

1. Dump the given .sql files (each file represents a single table) into your MySQL database.
2. Explore the provided tables and perform some level of EDA (either using Python libraries or using SQL query) to get insights about the data.*
3. Perform monthly and weekly level product wise aggregations of the transactions for each user and maintain both in separate tables.
4. Calculate all the facts required for customer profile table using any convenient tool (Python or SQL or Spark) utilizing the aggregate table created in step 3.
5. Create an Airflow DAG that will execute the script or code for steps 3 and 4 sequentially which can be scheduled as per the requirement (daily or hourly).


In [1]:
import pandas as pd

from helper import table_df

### Loading Tables

In [2]:
customer_profile = table_df('customer', 'customer_profile')

rw_transaction_data = table_df('customer', 'rw_transaction_data')
products = table_df('customer', 'products')
product_category = table_df('customer', 'product_category')
product_category_map = table_df('customer', 'product_category_map')

### Customer Profile

In [3]:
customer_profile.head()

,account_id,reward_point,total_inflow_amount,total_outflow_amount,total_valuechain_amount,total_inflow_count,total_outflow_count,total_valuechain_count,monthly_inflow_amount,monthly_outflow_amount,...,latest_used_product,latest_tran_date,this_month_revenue,monthly_average_lifetime_revenue,total_revenue,product_usage,most_used_product,second_most_used_product,third_most_used_product,run_date
0,1,1574.0,76385.0,70045.0,7650.0,39,31,4,76385.0,70045.0,...,Send Money,2081-01-26,31.0,31.0,31.0,14,Send Money,Fonepay Payment,eSewa to Laxmi Sunrise Bank Ltd.,2024-01-05
1,2,306.0,557075.0,184405.0,344798.0,753,142,400,16005.0,6180.0,...,Fonepay Payment,2081-01-31,4.0,62.0,2727.0,47,NT Prepaid Topup,Send Money,Ncell Topup,2024-01-05
2,3,10.0,18415.0,12400.0,5110.0,6,5,9,18415.0,12400.0,...,NT Prepaid Topup,2081-01-31,16.0,16.0,16.0,8,Ncell Topup,Send Money,Fonepay Payment,2024-01-05
3,4,324.0,6350.0,3050.0,7499.0,4,3,8,6350.0,3050.0,...,Fonepay Payment,2081-01-21,21.0,21.0,21.0,8,Fonepay Payment,Send Money,Electricity,2024-01-05
4,5,929.0,168576.0,100385.0,53911.0,69,24,95,106106.0,88660.0,...,Fonepay Payment,2081-01-31,70.0,28.0,1226.0,37,NT Prepaid Topup,NT Postpaid Topup,Send Money,2024-01-05


### Shape

In [4]:
names = ['rw_transaction_data', 'products', 'product_category', 'product_category_map']
tables= [rw_transaction_data, products, product_category, product_category_map]

for name, table in zip(names, tables):
    print(f'{name}\'shape: {table.shape}')

rw_transaction_data'shape: (11012, 16)
products'shape: (399, 5)
product_category'shape: (191, 8)
product_category_map'shape: (183, 6)


### Missing and Duplicates Values

In [5]:
# Check for missing values

for name, table in zip(names, tables):
    print(f'Missing values of {name}: {table.isna().sum().sum()}')
    print(f'Duplicated values of {name}: {table.duplicated().sum()}')

Missing values of rw_transaction_data: 0
Duplicated values of rw_transaction_data: 0
Missing values of products: 0
Duplicated values of products: 0
Missing values of product_category: 205
Duplicated values of product_category: 0
Missing values of product_category_map: 41
Duplicated values of product_category_map: 0


### Transaction Data

In [6]:
rw_transaction_data.head()

,txn_id,last_modified_date,last_modified_date_bs,created_date,amount,status,module_id,product_id,product_type_id,payer_account_id,receiver_account_id,reward_point,cash_back_amount,revenue_amount,transactor_module_id,time
0,660612529,2023-03-29,2079-12-15,2022-11-23,50.0,1,1,77,29,531,2,0.0,0.00,0.00,4,14:07:40
1,666435422,2022-12-01,2079-08-15,2022-12-01,750.0,1,1,76,29,531,81,0.0,0.00,33.75,4,00:03:41
2,666436001,2022-12-01,2079-08-15,2022-12-01,50.0,1,1,77,29,531,2,0.0,1.38,0.00,4,00:06:20
3,666436068,2022-12-01,2079-08-15,2022-12-01,50.0,1,1,77,29,531,647,0.0,1.38,0.00,4,00:06:32
4,666437220,2022-12-01,2079-08-15,2022-12-01,100.0,1,1,30,29,531,907,0.0,3.90,0.00,4,00:12:51


In [7]:
rw_transaction_data.shape

(11012, 16)

In [8]:
rw_transaction_data.nunique()

txn_id                   11012
last_modified_date           7
last_modified_date_bs        7
created_date                 7
amount                    1360
status                       1
module_id                    2
product_id                 145
product_type_id             23
payer_account_id             7
receiver_account_id        250
reward_point                 9
cash_back_amount           243
revenue_amount              87
transactor_module_id         2
time                      9884
dtype: int64

### Product and Product Category

In [9]:
products.head()

,module_id,product_id,product_type_id,product_name,product_type_name
0,1,77,29,Ncell Topup via Bank,Card Server Payment
1,1,76,29,SIM TV TOPUP VIA BANK,Card Server Payment
2,1,30,29,NT Topup via Bank,Card Server Payment
3,1,133,29,Wordlink Topup via Bank,Card Server Payment
4,1,89,29,Smart Cell Topup via Bank,Card Server Payment


In [10]:
products.shape

(399, 5)

In [11]:
product_category_map.head()

,module_id,product_id,product_type_id,product_name,product_category_id,txn_flow
0,1,670,11,"Siddhartha Sishu Sadan, Ln Chowk, Dhangadhi",33.0,Value Chain
1,2,2545,3,Multipurpose Finance Transfer,NaN,OutFlow
2,2,1444,6,Century Corporate Transfer,NaN,OutFlow
3,2,1008,7,Srijana Corporate Fund Receiver,NaN,OutFlow
4,2,885,10,Pokhara_Convergence,NaN,OutFlow


In [12]:
product_category.head()

,id,category_name,description,parent_id,status,active_flag,created_on,updated_on
0,1,Telecommunications,Telecommunications,NaN,0,1,2024-04-21 09:42:39,None
1,2,Utility,Utility,NaN,0,1,2024-04-21 09:42:39,None
2,3,TV-Internet-Movies,TV-Internet-Movies,NaN,0,1,2024-04-21 09:42:39,None
3,4,Travel and Airline Services,Travel and Airline Services,NaN,0,1,2024-04-21 09:42:39,None
4,5,Ride Sharing,Ride Sharing,NaN,0,1,2024-04-21 09:42:39,None


In [13]:
product_category.drop('updated_on', inplace=True, axis=1)

In [14]:
product_category.rename(columns={'id': 'product_category_id'}, inplace=True)

### Joins

In [15]:
merged_df_1 = pd.merge(rw_transaction_data, products, on = ['product_id', 'product_type_id', 'module_id'], how = 'inner')
merged_df_1.head()

,txn_id,last_modified_date,last_modified_date_bs,created_date,amount,status,module_id,product_id,product_type_id,payer_account_id,receiver_account_id,reward_point,cash_back_amount,revenue_amount,transactor_module_id,time,product_name,product_type_name
0,660612529,2023-03-29,2079-12-15,2022-11-23,50.0,1,1,77,29,531,2,0.0,0.00,0.00,4,14:07:40,Ncell Topup via Bank,Card Server Payment
1,666435422,2022-12-01,2079-08-15,2022-12-01,750.0,1,1,76,29,531,81,0.0,0.00,33.75,4,00:03:41,SIM TV TOPUP VIA BANK,Card Server Payment
2,666436001,2022-12-01,2079-08-15,2022-12-01,50.0,1,1,77,29,531,2,0.0,1.38,0.00,4,00:06:20,Ncell Topup via Bank,Card Server Payment
3,666436068,2022-12-01,2079-08-15,2022-12-01,50.0,1,1,77,29,531,647,0.0,1.38,0.00,4,00:06:32,Ncell Topup via Bank,Card Server Payment
4,666437220,2022-12-01,2079-08-15,2022-12-01,100.0,1,1,30,29,531,907,0.0,3.90,0.00,4,00:12:51,NT Topup via Bank,Card Server Payment


In [16]:
merged_df_1.shape

(11012, 18)

In [17]:
merged_df_1.nunique()

txn_id                   11012
last_modified_date           7
last_modified_date_bs        7
created_date                 7
amount                    1360
status                       1
module_id                    2
product_id                 145
product_type_id             23
payer_account_id             7
receiver_account_id        250
reward_point                 9
cash_back_amount           243
revenue_amount              87
transactor_module_id         2
time                      9884
product_name               144
product_type_name           24
dtype: int64

In [18]:
product_category_map.head()

,module_id,product_id,product_type_id,product_name,product_category_id,txn_flow
0,1,670,11,"Siddhartha Sishu Sadan, Ln Chowk, Dhangadhi",33.0,Value Chain
1,2,2545,3,Multipurpose Finance Transfer,NaN,OutFlow
2,2,1444,6,Century Corporate Transfer,NaN,OutFlow
3,2,1008,7,Srijana Corporate Fund Receiver,NaN,OutFlow
4,2,885,10,Pokhara_Convergence,NaN,OutFlow


In [19]:
product_category_map_dropped = product_category_map.drop(['product_name', 'product_category_id'], axis = 1)

In [20]:
merged_df_2 = pd.merge(merged_df_1, product_category_map_dropped, on = ['product_id', 'product_type_id', 'module_id'], how='inner')

In [21]:
merged_df_2.head()

,txn_id,last_modified_date,last_modified_date_bs,created_date,amount,status,module_id,product_id,product_type_id,payer_account_id,receiver_account_id,reward_point,cash_back_amount,revenue_amount,transactor_module_id,time,product_name,product_type_name,txn_flow
0,660612529,2023-03-29,2079-12-15,2022-11-23,50.0,1,1,77,29,531,2,0.0,0.00,0.00,4,14:07:40,Ncell Topup via Bank,Card Server Payment,Value Chain
1,660612529,2023-03-29,2079-12-15,2022-11-23,50.0,1,1,77,29,531,2,0.0,0.00,0.00,4,14:07:40,Ncell Topup via Bank,Card Server Payment,Value Chain
2,666435422,2022-12-01,2079-08-15,2022-12-01,750.0,1,1,76,29,531,81,0.0,0.00,33.75,4,00:03:41,SIM TV TOPUP VIA BANK,Card Server Payment,Value Chain
3,666436001,2022-12-01,2079-08-15,2022-12-01,50.0,1,1,77,29,531,2,0.0,1.38,0.00,4,00:06:20,Ncell Topup via Bank,Card Server Payment,Value Chain
4,666436001,2022-12-01,2079-08-15,2022-12-01,50.0,1,1,77,29,531,2,0.0,1.38,0.00,4,00:06:20,Ncell Topup via Bank,Card Server Payment,Value Chain


In [22]:
# merged_df_2.rename(columns={'product_name_x': 'product_name'}, inplace=True)

In [23]:
merged_df_2.shape

(10780, 19)

In [24]:
merged_df_2.nunique()

txn_id                   9018
last_modified_date          7
last_modified_date_bs       7
created_date                7
amount                    478
status                      1
module_id                   2
product_id                132
product_type_id            23
payer_account_id            7
receiver_account_id       237
reward_point                9
cash_back_amount          240
revenue_amount             87
transactor_module_id        2
time                     8278
product_name              132
product_type_name          24
txn_flow                    3
dtype: int64

In [25]:
# df = pd.merge(merged_df_2, product_category, on = ['product_category_id'], how='left')
# df.head()

In [26]:
df = merged_df_2.copy()
df.duplicated().sum()

1607

In [27]:
df.drop_duplicates(inplace = True)

In [28]:
df.shape

(9173, 19)

In [29]:
df.head()

,txn_id,last_modified_date,last_modified_date_bs,created_date,amount,status,module_id,product_id,product_type_id,payer_account_id,receiver_account_id,reward_point,cash_back_amount,revenue_amount,transactor_module_id,time,product_name,product_type_name,txn_flow
0,660612529,2023-03-29,2079-12-15,2022-11-23,50.0,1,1,77,29,531,2,0.0,0.00,0.00,4,14:07:40,Ncell Topup via Bank,Card Server Payment,Value Chain
2,666435422,2022-12-01,2079-08-15,2022-12-01,750.0,1,1,76,29,531,81,0.0,0.00,33.75,4,00:03:41,SIM TV TOPUP VIA BANK,Card Server Payment,Value Chain
3,666436001,2022-12-01,2079-08-15,2022-12-01,50.0,1,1,77,29,531,2,0.0,1.38,0.00,4,00:06:20,Ncell Topup via Bank,Card Server Payment,Value Chain
5,666436068,2022-12-01,2079-08-15,2022-12-01,50.0,1,1,77,29,531,647,0.0,1.38,0.00,4,00:06:32,Ncell Topup via Bank,Card Server Payment,Value Chain
7,666437220,2022-12-01,2079-08-15,2022-12-01,100.0,1,1,30,29,531,907,0.0,3.90,0.00,4,00:12:51,NT Topup via Bank,Card Server Payment,Value Chain


In [30]:
# ID to String

df[['txn_id', 'module_id', 'product_id', 'product_type_id', 'payer_account_id', 'receiver_account_id', 'transactor_module_id']] = \
    df[['txn_id', 'module_id', 'product_id', 'product_type_id', 'payer_account_id', 'receiver_account_id', 'transactor_module_id']].apply(lambda x: x.astype(str))

In [31]:
df['last_modified_date'] = df['last_modified_date'].astype(str) 

In [32]:
df['dates'] = pd.to_datetime(df['last_modified_date'] + df['time'], format='%Y-%m-%d%H:%M:%S')

In [33]:
df[['last_modified_date', 'time', 'dates']]

,last_modified_date,time,dates
0,2023-03-29,14:07:40,2023-03-29 14:07:40
2,2022-12-01,00:03:41,2022-12-01 00:03:41
3,2022-12-01,00:06:20,2022-12-01 00:06:20
5,2022-12-01,00:06:32,2022-12-01 00:06:32
7,2022-12-01,00:12:51,2022-12-01 00:12:51
...,...,...,...
10775,2023-01-06,11:37:20,2023-01-06 11:37:20
10776,2023-01-06,11:37:36,2023-01-06 11:37:36
10777,2023-01-06,11:37:48,2023-01-06 11:37:48
10778,2023-01-06,11:37:52,2023-01-06 11:37:52


In [34]:
# Date 
# df['last_modified_date'] = pd.to_datetime(df['last_modified_date'], errors = 'coerce') 
df[['last_modified_date', 'last_modified_date_bs', 'created_date']] = df[['last_modified_date', 'last_modified_date_bs', 'created_date']].apply(pd.to_datetime, errors='coerce')
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time

# Creating monthly and weekly columns
df['monthly'] = df['last_modified_date'].dt.month
df['weekly'] = df['last_modified_date'].dt.isocalendar().week


### TXN Flow

In [36]:
pivot_monthly = pd.pivot_table(df, index = ['payer_account_id', 'monthly'], columns = 'txn_flow', values = 'amount', aggfunc = 'sum').reset_index()
pivot_monthly

txn_flow,payer_account_id,monthly,InFlow,OutFlow,Value Chain
0,1056,12,NaN,NaN,200.0
1,222,1,NaN,NaN,120.0
2,222,12,NaN,NaN,135.0
3,26,1,34340.0,51340.0,16511.0
4,26,12,NaN,NaN,280.0
5,34,1,2133150.0,2396770.0,239587.0
6,34,12,83200.0,88200.0,3251.0
7,471,1,700.0,700.0,12425.0
8,471,12,NaN,NaN,1524.0
9,531,1,NaN,NaN,1264668.0


In [37]:
pivot_monthly_count = pd.pivot_table(df, index = ['payer_account_id', 'monthly'], columns = 'txn_flow', values = 'amount', aggfunc = 'count').reset_index()
pivot_monthly_count 

txn_flow,payer_account_id,monthly,InFlow,OutFlow,Value Chain
0,1056,12,NaN,NaN,1.0
1,222,1,NaN,NaN,1.0
2,222,12,NaN,NaN,1.0
3,26,1,12.0,15.0,23.0
4,26,12,NaN,NaN,6.0
5,34,1,121.0,137.0,58.0
6,34,12,5.0,6.0,4.0
7,471,1,2.0,2.0,52.0
8,471,12,NaN,NaN,10.0
9,531,1,NaN,NaN,6082.0


In [38]:
aggregrated_df_monthly = (
    pivot_monthly
    .groupby('payer_account_id')
    .agg(
        total_inflow_amount = ('InFlow', 'sum'),
        total_outflow_amount = ('OutFlow', 'sum'),
        total_valuechain_amount = ('Value Chain', 'sum'),
        monthly_inflow_amount = ('InFlow', 'mean'),
        monthly_outflow_amount = ('OutFlow', 'mean'),
        monthly_valuechain_amount = ('Value Chain', 'mean'),
    )
    .reset_index()
)

In [39]:
aggregrated_df_monthly_count = (
    pivot_monthly_count
    .groupby('payer_account_id')
    .agg(
        total_inflow_count = ('InFlow', 'sum'),
        total_outflow_count = ('OutFlow', 'sum'),
        total_valuechain_count = ('Value Chain', 'sum'),
        monthly_inflow_count = ('InFlow', 'mean'),
        monthly_outflow_count = ('OutFlow', 'mean'),
        monthly_valuechain_count = ('Value Chain', 'mean'),
    )
    .reset_index()
)

In [40]:
aggregrated_df_monthly

,payer_account_id,total_inflow_amount,total_outflow_amount,total_valuechain_amount,monthly_inflow_amount,monthly_outflow_amount,monthly_valuechain_amount
0,1056,0.0,0.0,200.0,NaN,NaN,200.000000
1,222,0.0,0.0,255.0,NaN,NaN,127.500000
2,26,34340.0,51340.0,16791.0,34340.0,51340.0,8395.500000
3,34,2216350.0,2484970.0,242838.0,1108175.0,1242485.0,121419.000000
4,471,700.0,700.0,13949.0,700.0,700.0,6974.500000
5,531,0.0,0.0,1720225.0,NaN,NaN,573408.333333
6,56,25475.0,147875.0,79081.0,12737.5,73937.5,39540.500000


In [41]:
aggregrated_df_monthly_count

,payer_account_id,total_inflow_count,total_outflow_count,total_valuechain_count,monthly_inflow_count,monthly_outflow_count,monthly_valuechain_count
0,1056,0.0,0.0,1.0,NaN,NaN,1.000000
1,222,0.0,0.0,2.0,NaN,NaN,1.000000
2,26,12.0,15.0,29.0,12.0,15.0,14.500000
3,34,126.0,143.0,62.0,63.0,71.5,31.000000
4,471,2.0,2.0,62.0,2.0,2.0,31.000000
5,531,0.0,0.0,8548.0,NaN,NaN,2849.333333
6,56,15.0,22.0,132.0,7.5,11.0,66.000000


### Reward Points

In [35]:
reward_agg = df.groupby('payer_account_id').agg({'reward_point': 'sum'})
reward_agg

,reward_point
payer_account_id,
1056,4.0
222,0.0
26,0.0
34,68.0
471,14.0
531,0.0
56,39.0


### Lastest Transaction Date and Used Product

In [42]:
idx_max_date = df.groupby('payer_account_id')['dates'].idxmax()
result_df = df.loc[idx_max_date, ['payer_account_id', 'product_name', 'dates']]
result_df.rename(columns = {'product_name': 'latest_used_product', 'dates': 'latest_tran_date'}, inplace=True)
result_df

,payer_account_id,latest_used_product,latest_tran_date
3728,1056,NT Prepaid Topup,2022-12-01 21:04:02
4168,222,Ncell Topup,2023-01-01 09:39:35
10719,26,NT Prepaid Topup,2023-01-06 11:13:47
10767,34,Nepal Electricity Authority,2023-01-06 11:34:40
10731,471,NT Prepaid Topup,2023-01-06 11:17:50
0,531,Ncell Topup via Bank,2023-03-29 14:07:40
10772,56,Nepal Electricity Authority,2023-01-06 11:36:30


### Revenue

In [43]:
monthly_revenue= pd.pivot_table(df, index = ['payer_account_id', 'monthly'], values = 'revenue_amount', aggfunc = 'sum').reset_index()
monthly_revenue

,payer_account_id,monthly,revenue_amount
0,1056,12,7.00
1,222,1,0.90
2,222,12,1.01
3,26,1,37.14
4,26,12,0.59
5,34,1,491.33
6,34,12,16.10
7,471,1,64.17
8,471,12,5.72
9,531,1,1768.23


In [44]:
revenue = (
    monthly_revenue
    .groupby('payer_account_id')
    .agg(
        monthly_average_lifetime_revenue = ('revenue_amount', 'mean'),
        total_revenue = ('revenue_amount', 'sum'),
    )
    .reset_index()
)

revenue

,payer_account_id,monthly_average_lifetime_revenue,total_revenue
0,1056,7.000000,7.00
1,222,0.955000,1.91
2,26,18.865000,37.73
3,34,253.715000,507.43
4,471,34.945000,69.89
5,531,758.013333,2274.04
6,56,89.295000,178.59


### This Month's Revenue

In [51]:
latest_date = df['dates'].max()
latest_month_df = df[df['dates'].dt.month == latest_date.month]

this_month_revenue = latest_month_df.groupby('payer_account_id')['revenue_amount'].sum().reset_index(name = 'this_month_revenue')
this_month_revenue

,payer_account_id,this_month_revenue
0,531,0.0


### Product Usage

In [45]:
product_usage = df.groupby('payer_account_id')['product_id'].count().reset_index().rename(columns={'product_id': 'product_usage'})
product_usage

,payer_account_id,product_usage
0,1056,1
1,222,2
2,26,56
3,34,331
4,471,66
5,531,8548
6,56,169


### Nth Used Product

In [46]:
product_counts = df.groupby(['payer_account_id', 'product_name'])['product_name'].count().reset_index(name='count')

product_counts

,payer_account_id,product_name,count
0,1056,NT Prepaid Topup,1
1,222,Ncell Topup,2
2,26,Cash In,10
3,26,Fonepay Payment,2
4,26,NT FTTH,1
...,...,...,...
151,56,WorldLink,8
152,56,eScrow Service,6
153,56,eSewa to Citizens Bank,5
154,56,eSewa to NIC Asia Bank,1


In [47]:
sorted_product_counts = product_counts.sort_values(by=['payer_account_id', 'count'], ascending=[True, False])

most_used_product = sorted_product_counts.groupby('payer_account_id').nth(0).reset_index()
second_most_used_product = sorted_product_counts.groupby('payer_account_id').nth(1).reset_index()
third_most_used_product = sorted_product_counts.groupby('payer_account_id').nth(2).reset_index()

In [48]:
most_used_product 

,index,payer_account_id,product_name,count
0,0,1056,NT Prepaid Topup,1
1,1,222,Ncell Topup,2
2,6,26,NT Prepaid Topup,18
3,32,34,Send Money,248
4,48,471,NT Prepaid Topup,32
5,101,531,NT Topup via Bank,6495
6,148,56,Nepal Electricity Authority,52


In [49]:
result = pd.merge(most_used_product, second_most_used_product, on='payer_account_id', suffixes=('_most', '_second'), how='left')
result = pd.merge(result, third_most_used_product, on='payer_account_id', how='left')

In [50]:
result = result[['payer_account_id', 'product_name_most', 'product_name_second', 'product_name']]
result.columns = ['payer_account_id', 'most_used_product', 'second_most_used_product', 'third_most_used_product']
result

,payer_account_id,most_used_product,second_most_used_product,third_most_used_product
0,1056,NT Prepaid Topup,NaN,NaN
1,222,Ncell Topup,NaN,NaN
2,26,NT Prepaid Topup,Send Money,Cash In
3,34,Send Money,Nepal Electricity Authority,NT Prepaid Topup
4,471,NT Prepaid Topup,Ncell Topup,Dish Home Topup
5,531,NT Topup via Bank,Ncell Topup via Bank,Wordlink Topup via Bank
6,56,Nepal Electricity Authority,Ncell Topup,Send Money
